# ACCRE Project - Data Import and Cleaning

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
import matplotlib.dates as mdates
import re

In [2]:
# Read in fullsample.csv as a dataframe
jobs = pd.read_csv("../data/fullsample.csv")

In [3]:
# Convert BEGIN and END columns to datetime type
jobs['BEGIN'] = pd.to_datetime(jobs['BEGIN'], errors='coerce')
jobs['END'] = pd.to_datetime(jobs['END'], errors='coerce')

# Drop all columns with null type in BEGIN or END columns
jobs = jobs.dropna(subset=['BEGIN', 'END'])

# Calculate length of jobs and assign in new column JOBLENGTH
jobs['JOBLENGTH'] = jobs['END'] - jobs['BEGIN']

# Convert REQTIME and USEDTIME columns to timedelta type
jobs['REQTIME'] = pd.to_timedelta(jobs['REQTIME'])
jobs['USEDTIME'] = pd.to_timedelta(jobs['USEDTIME'])

In [4]:
# Calculate total memory and memory per core for each job
for row_tuple in jobs.itertuples():
    memory = row_tuple[5]
    memory_int = int(re.findall(r'\d+', memory)[0]) 
    nodes = row_tuple[9] 
    cpus = row_tuple[10]
    
    if 'Mn' in memory:
        total_memory = memory_int * nodes
        jobs.at[row_tuple[0], 'REQMEMTOT'] = total_memory
        if cpus == 0:
            jobs.at[row_tuple[0], 'REQMEMPERCORE'] = total_memory
        else:
            jobs.at[row_tuple[0], 'REQMEMPERCORE'] = total_memory / cpus
    elif 'Mc' in memory:
        total_memory = memory_int * cpus
        jobs.at[row_tuple[0], 'REQMEMTOT'] = total_memory
        jobs.at[row_tuple[0], 'REQMEMPERCORE'] = memory_int

In [5]:
# Export cleaned dataframe as a csv
jobs.to_csv('../data/fullsample_cleaned.csv', index=False)